Convert pyspark string to date format issue dd-mm-yy old format

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
schema_channels = StructType([StructField('CHANNEL_ID', IntegerType(), True), 
                              StructField('CHANNEL_DESC', StringType(), True), 
                              StructField('CHANNEL_CLASS', StringType(), True), 
                              StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
                              StructField('CHANNEL_TOTAL', StringType(), True), 
                              StructField('CHANNEL_TOTAL_ID', IntegerType(), True)])

Permissive Mode (default)

In [4]:
df_channels = spark.read\
    .option("mode", "PERMISSIVE")\
    .csv("/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/bad_records.csv",header=True, inferSchema=True)

df_channels.show(truncate=False)

+-------------------------------------+------------+-------------+----------------+-------------+----------------+
|CHANNEL_ID                           |CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+-------------------------------------+------------+-------------+----------------+-------------+----------------+
|3                                    |Direct Sales|Direct       |12              |Channel total|1               |
|9                                    |Tele Sales  |Direct       |12              |Channel total|1               |
|5                                    |Catalog     |Indirect     |13              |Channel total|1               |
|4                                    |Internet    |Indirect     |13              |Channel total|1               |
|2                                    |Partners    |Others       |14              |Channel total|1               |
|12                                   |Partners    |Others       |14            

Permissive With Schema

In [5]:
df_channels = spark.read\
    .schema(schema_channels)\
    .option("mode", "PERMISSIVE")\
    .csv("/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/bad_records.csv",header=True, inferSchema=True)

df_channels.show(truncate=False)

+----------+------------+-------------+----------------+-------------+----------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+----------+------------+-------------+----------------+-------------+----------------+
|3         |Direct Sales|Direct       |12              |Channel total|1               |
|9         |Tele Sales  |Direct       |12              |Channel total|1               |
|5         |Catalog     |Indirect     |13              |Channel total|1               |
|4         |Internet    |Indirect     |13              |Channel total|1               |
|2         |Partners    |Others       |14              |Channel total|1               |
|12        |Partners    |Others       |14              |Channel total|1               |
|null      |Partners    |Others       |14              |Channel total|1               |
|null      |null        |null         |null            |null         |null            |
|null      |null        |null   

Fail Fast Mode

In [6]:
df_channels = spark.read\
    .schema(schema_channels)\
    .option("mode", "FAILFAST")\
    .csv("/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/bad_records.csv",header=True, inferSchema=True)

df_channels.show(truncate=False)

23/05/22 18:41:19 ERROR Executor: Exception in task 0.0 in stage 4.0 (TID 4)
org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1417)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:421)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catal

Py4JJavaError: An error occurred while calling o60.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4) (172.25.124.89 executor driver): org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1417)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:68)
	at org.apache.spark.sql.catalyst.csv.UnivocityParser$.$anonfun$parseIterator$2(UnivocityParser.scala:421)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.

Mode DROPMALFORMED

In [7]:
df_channels = spark.read\
    .schema(schema_channels)\
    .option("mode", "DROPMALFORMED")\
    .csv("/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/bad_records.csv",header=True, inferSchema=True)

df_channels.show(truncate=False)

+----------+------------+-------------+----------------+-------------+----------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+----------+------------+-------------+----------------+-------------+----------------+
|3         |Direct Sales|Direct       |12              |Channel total|1               |
|9         |Tele Sales  |Direct       |12              |Channel total|1               |
|5         |Catalog     |Indirect     |13              |Channel total|1               |
|4         |Internet    |Indirect     |13              |Channel total|1               |
|2         |Partners    |Others       |14              |Channel total|1               |
+----------+------------+-------------+----------------+-------------+----------------+



badRecordsPath

In [39]:
df_channels = spark.read\
    .schema(schema_channels)\
    .option("badRecordsPath", "/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/bad/")\
    .csv("/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/bad_records.csv",header=True)

df_channels.show(truncate=False)

+----------+------------+-------------+----------------+-------------+----------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+----------+------------+-------------+----------------+-------------+----------------+
|3         |Direct Sales|Direct       |12              |Channel total|1               |
|9         |Tele Sales  |Direct       |12              |Channel total|1               |
|5         |Catalog     |Indirect     |13              |Channel total|1               |
|4         |Internet    |Indirect     |13              |Channel total|1               |
|2         |Partners    |Others       |14              |Channel total|1               |
|12        |Partners    |Others       |14              |Channel total|1               |
|null      |Partners    |Others       |14              |Channel total|1               |
|null      |null        |null         |null            |null         |null            |
|null      |null        |null   

columnNameOfCorruptRecord

In [48]:
schema_channels_bad_data = StructType([StructField('CHANNEL_ID', IntegerType(), True), 
                              StructField('CHANNEL_DESC', StringType(), True), 
                              StructField('CHANNEL_CLASS', StringType(), True), 
                              StructField('CHANNEL_CLASS_ID', IntegerType(), True), 
                              StructField('CHANNEL_TOTAL', StringType(), True), 
                              StructField('CHANNEL_TOTAL_ID', IntegerType(), True),
                              StructField('BadData', StringType(), False)])

In [49]:

df_channels = spark.read\
    .schema(schema_channels_bad_data)\
    .csv("/home/phillipefs/spark_dev/pyspark-real-time-scenarios/data/bad_records.csv",header=True,columnNameOfCorruptRecord="BadData")

df_channels.show(truncate=False)

+----------+------------+-------------+----------------+-------------+----------------+-----------------------------------------------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|BadData                                              |
+----------+------------+-------------+----------------+-------------+----------------+-----------------------------------------------------+
|3         |Direct Sales|Direct       |12              |Channel total|1               |null                                                 |
|9         |Tele Sales  |Direct       |12              |Channel total|1               |null                                                 |
|5         |Catalog     |Indirect     |13              |Channel total|1               |null                                                 |
|4         |Internet    |Indirect     |13              |Channel total|1               |null                                                 |
|2    

In [73]:
df_channels.cache()
df_good = df_channels.filter("BadData is null").drop("BadData")
df_bad = df_channels.filter("BadData is not null").select("BadData")

In [75]:
df_good.show()

+----------+------------+-------------+----------------+-------------+----------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+----------+------------+-------------+----------------+-------------+----------------+
|         3|Direct Sales|       Direct|              12|Channel total|               1|
|         9|  Tele Sales|       Direct|              12|Channel total|               1|
|         5|     Catalog|     Indirect|              13|Channel total|               1|
|         4|    Internet|     Indirect|              13|Channel total|               1|
|         2|    Partners|       Others|              14|Channel total|               1|
+----------+------------+-------------+----------------+-------------+----------------+



In [74]:
df_bad.show(truncate=False)

+-----------------------------------------------------+
|BadData                                              |
+-----------------------------------------------------+
|12,Partners,Others,14,Channel total,1,45,ram,3434    |
|sample,Partners,Others,14,Channel total,1,45,ram,3434|
|10 Partners Others 14 Channel total 1                |
|11 Partners Others 14 Channel total 1                |
+-----------------------------------------------------+

